In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

#### Goal: we want to create a PartitionSampler

The sampler samples ordered n-tuples $a₁≤a₂≤…≤aₙ$

Additionally we want to satisfy constraints:


- outer bounds: given $l₁≤l₂≤…≤lₙ$ and $u₁≤u₂≤…≤uₙ$ such that $lᵢ≤uᵢ$ we require $lᵢ≤aᵢ≤uᵢ$
- distance bounds: we require $lᵢ≤|aᵢ₊₁ - aᵢ|≤uᵢ$

Uniform distribution over grids:

- We assume each $aᵢ$ lies on a regular grid with step-size $∆ᵢ$ and anchor point $aᵢ^*$.
- How do we get all valid samples?





#### Example: Sampling observation and forecasting horizons.

During training we want to test whether it helps to sample observation and forecasting horizons of varying length

### Goal: We want to avoid combinatorial explosion, otherwise the sampler get large  (at least in the default configuration!)

Therefore, we make in the default settings some simplifications:

- all points lie on the same grid.
- one option is to keep the total size constant ⇝ nice for batching?!
- deltas are a multiple of a base delta


Solution: recursion! Assume we have a sampler that works for n points.
Then we construct a sampler that works for n+1 points.

### Special Case: observation + forecasting horizon (triplet $a<b<c$)

These are the most important cases from which we derive the generalization.

- Fixed observation and forecasting horizon: $c-b = \text{const.}$ and $b-a = \text{const.}$
    - Either "real time horizon" or inter number of datapoints
    - Simple move the whole tuple forward by stride as long as its possible.
    - Sampling grid:  $\{(t_k - ∆t_\text{obs}, t_{k}, t_{k}+ ∆t_\text{pred}) ∣ t_k = t_\text{start} + k⋅∆t_\text{grid}\}$
- Varying size observation horizon
    - Observe all data since start of experiment 
    - Sampling grid:  $\{(t_\text{start}, t_{k}, t_{k}+∆t_\text{pred})∣ t_k = t_\text{start} +t_\text{obs} + k⋅∆t_\text{grid}\}$
- Varying size forecasting horizon
    - Forecast for the remaining experiment time (i.e. a-priori fixed $t_\text{final}$)
    - Forecast for 1h, for 2h, for 4h ⟹ aggregate results / given trade-off?
    - Sampling grid:  $\{(t_\text{k}-∆t_\text{obs}, t_\text{k}, t_\text{final}) ∣ t_k = t_\text{start} + k⋅∆t_\text{grid} \}$
- Varying observation + forecasting horizon
    - Sampling grid: $\{(t_\text{start}, t_\text{k}, t_\text{final}) ∣ t_k = t_\text{start} + k⋅∆t_\text{grid} \}$




Note that in all of these cases can be seen as generalizations of

$$(a_k, b_k, c_k) = (a_0 + k⋅∆a, b_0 + k⋅∆b, c_0 + k⋅∆c)$$

1. Case $\{(t_k - ∆t_\text{obs}, t_{k}, t_{k}+ ∆t_\text{pred}) ∣ t_k = t_\text{start} + k⋅∆t_\text{grid}\}$
    - $a_0=t_\text{start}$
    - $b_0=t_\text{start}+∆t_\text{obs}$
    - $c_0=t_\text{start}+∆t_\text{obs}+ ∆t_\text{pred}$
    - $∆a=∆b=∆c=∆t_\text{grid}$
2. Case $\{(t_\text{start}, t_{k}, t_{k}+∆t_\text{pred})∣ t_k = t_\text{start} +t_\text{obs}+ k⋅∆t_\text{grid}\}$
    - $a_0=t_\text{start}$
    - $b_0=t_\text{start}+∆t_\text{obs}$
    - $c_0=t_\text{start}+∆t_\text{obs}+ ∆t_\text{pred}$
    - $∆a=0$, $∆b=∆c=∆t_\text{grid}$
3. Case $\{(t_\text{k}-∆t_\text{obs}, t_\text{k}, t_\text{final}) ∣ t_k = t_\text{start} + k⋅∆t_\text{grid} \}$
    - $a_0=t_\text{start}$
    - $b_0=t_\text{start}+∆t_\text{obs}$
    - $c_0=t_\text{final}$
    - $∆a=∆b=∆t_\text{grid}$, $∆c=0$
4. Case $\{(t_\text{start}, t_\text{k}, t_\text{final}) ∣ t_k = t_\text{start} + k⋅∆t_\text{grid} \}$
    - $a_0=t_\text{start}$
    - $b_0=t_\text{start}+∆t_\text{obs}$
    - $c_0=t_\text{final}$
    - $∆a=∆c=0$, $∆b=∆t_\text{grid}$
    
Note that here either $∆t=0$, or $∆t$ is the same for $a,b,c$. otherwise, there is potential for one counter to overtake the other!.
These 4 cases also do not cover the case when we can to consider multiple counters of different length.

In this case, one satisfying solution is to "concatenate" two samplers.